## Functions to derive DLS par score in T20 Matches

### Importing Libraries

In [1]:
import pandas as pd
import numpy as np

### Fetching DLS resource data

In [2]:
resource_table_df = pd.read_csv(f"dls_resource_data_for_t20s.csv")
resource_table_df

,balls,0,1,2,3,4,5,6,7,8,9
0,120,100.0,96.8,92.6,86.7,78.8,68.2,54.4,37.5,21.3,8.3
1,114,96.1,93.3,89.2,83.9,76.7,66.6,53.5,37.3,21.0,8.3
2,108,92.2,89.6,85.9,81.1,74.2,65.0,52.7,36.9,21.0,8.3
3,102,88.2,85.7,82.5,77.9,71.7,63.3,51.6,36.6,21.0,8.3
4,96,84.1,81.8,79.0,74.7,69.1,61.3,50.4,36.2,20.8,8.3
5,90,79.9,77.9,75.3,71.6,66.4,59.2,49.1,35.7,20.8,8.3
6,84,75.4,73.7,71.4,68.0,63.4,56.9,47.7,35.2,20.8,8.3
7,78,71.0,69.4,67.3,64.5,60.4,54.4,46.1,34.5,20.7,8.3
8,72,66.4,65.0,63.3,60.6,57.1,51.9,44.3,33.6,20.5,8.3
9,66,61.7,60.4,59.0,56.7,53.7,49.1,42.4,32.7,20.3,8.3


### function to convert from overs to balls

In [3]:
def convert_overs_to_balls(overs):
    integer_part = int(overs)
    decimal_part = overs - integer_part
    balls = integer_part * 6 + round(decimal_part * 10)
    return balls

### function to convert from balls to overs

In [4]:
def convert_balls_to_overs(balls):
    overs = balls//6 + balls%6 * 0.1
    return overs

### Scenario 1 --> Finding the parscore when First Innings is completed and Second innings is interrupted in the Middle

In [5]:
def get_par_score_when_first_innings_is_completed_and_second_innings_is_interrupted(
    overs_available_to_team_one,
    runs_scored_by_team_one,
    overs_available_to_team_two_at_start,
    overs_used_by_team_two_until_interruption,
    wickets_lost_by_team_two,
    maximum_overs_allotted_to_team_two_after_resumption
):
    resource_for_0_wickets_lost = resource_table_df["0"][::-1]
    
    balls_available_to_team_one_at_start = convert_overs_to_balls(overs_available_to_team_one)
    balls_available_to_team_two_at_start = convert_overs_to_balls(overs_available_to_team_two_at_start)
    balls_used_by_team_two_until_interruption = convert_overs_to_balls(overs_used_by_team_two_until_interruption)
    balls_remaining_to_team_two_during_interruption = balls_available_to_team_two_at_start - balls_used_by_team_two_until_interruption
    maximum_balls_alloted_to_team_two_after_resumption = convert_overs_to_balls(maximum_overs_allotted_to_team_two_after_resumption)
    balls_remaining_to_team_two_after_resumption = maximum_balls_alloted_to_team_two_after_resumption - balls_used_by_team_two_until_interruption
    
    total_resource_available_to_team_one_at_start = np.interp(
        balls_available_to_team_one_at_start, resource_table_df["balls"][::-1], resource_for_0_wickets_lost
    )
    total_resource_available_to_team_two_at_start = np.interp(
        balls_available_to_team_two_at_start, resource_table_df["balls"][::-1], resource_for_0_wickets_lost
    )
    
    team_two_remaining_resource_during_interruption = np.interp(
        balls_remaining_to_team_two_during_interruption,
        resource_table_df["balls"][::-1],
        resource_table_df[f"{wickets_lost_by_team_two}"][::-1]
    )
    team_two_remaining_resource_during_resumption = np.interp(
        balls_remaining_to_team_two_after_resumption,
        resource_table_df["balls"][::-1],
        resource_table_df[f"{wickets_lost_by_team_two}"][::-1]
    )
    
    resource_lost_by_team_two_due_to_interruption = team_two_remaining_resource_during_interruption - team_two_remaining_resource_during_resumption
    total_resource_remaining_to_team_two = total_resource_available_to_team_two_at_start - resource_lost_by_team_two_due_to_interruption
    
    par_score = runs_scored_by_team_one * (total_resource_remaining_to_team_two/total_resource_available_to_team_one_at_start)
    
    return par_score


In [6]:
get_par_score_when_first_innings_is_completed_and_second_innings_is_interrupted(
20, 214, 20, 0.3, 0, 15
)

170.34400000000002

### Scenario 2 --> Finding the parscore when First Innings is completed and Second innings is cut short in the Middle

Example - IRE vs ENG - T20WC 22 League match

In [7]:
def get_par_score_when_first_innings_is_completed_and_second_innings_is_cut_short(
    overs_available_to_team_one,
    runs_scored_by_team_one,
    overs_available_to_team_two_at_start,
    overs_used_by_team_two_until_cutoff,
    wickets_lost_by_team_two
):
    resource_for_0_wickets_lost = resource_table_df["0"][::-1]
    
    balls_available_to_team_one_at_start = convert_overs_to_balls(overs_available_to_team_one)
    balls_available_to_team_two_at_start = convert_overs_to_balls(overs_available_to_team_two_at_start)
    balls_used_by_team_two_until_cutoff = convert_overs_to_balls(overs_used_by_team_two_until_cutoff)
    balls_remaining_to_team_two = balls_available_to_team_two_at_start - balls_used_by_team_two_until_cutoff
    
    total_resource_available_to_team_one_at_start = np.interp(
        balls_available_to_team_one_at_start, resource_table_df["balls"][::-1], resource_for_0_wickets_lost
    )
    total_resource_available_to_team_two_at_start = np.interp(
        balls_available_to_team_two_at_start, resource_table_df["balls"][::-1], resource_for_0_wickets_lost
    )
    
    team_two_remaining_resource = np.interp(
        balls_remaining_to_team_two,
        resource_table_df["balls"][::-1],
        resource_table_df[f"{wickets_lost_by_team_two}"][::-1]
    )
    
    team_two_available_resource = total_resource_available_to_team_two_at_start - team_two_remaining_resource
    
    par_score = runs_scored_by_team_one * (team_two_available_resource/total_resource_available_to_team_one_at_start)
    
    return par_score


In [8]:
get_par_score_when_first_innings_is_completed_and_second_innings_is_cut_short(
20, 157, 20, 14.3, 5
)

110.99900000000001

### Scenario 3 --> Finding the parscore when First Innings is completed and Second innings is delayed

Example - WI vs BANGLA - 1st T20I 2018
https://www.espncricinfo.com/series/bangladesh-in-wi-and-usa-2018-1146712/west-indies-vs-bangladesh-1st-t20i-1146723/ball-by-ball-commentary

In [9]:
def get_par_score_when_first_innings_is_completed_and_second_innings_is_delayed(
    overs_available_to_team_one,
    runs_scored_by_team_one,
    overs_available_to_team_two_at_start
):
    resource_for_0_wickets_lost = resource_table_df["0"][::-1]
    
    balls_available_to_team_one_at_start = convert_overs_to_balls(overs_available_to_team_one)
    balls_available_to_team_two_at_start = convert_overs_to_balls(overs_available_to_team_two_at_start)
    
    total_resource_available_to_team_one_at_start = np.interp(
        balls_available_to_team_one_at_start, resource_table_df["balls"][::-1], resource_for_0_wickets_lost
    )
    total_resource_available_to_team_two_at_start = np.interp(
        balls_available_to_team_two_at_start, resource_table_df["balls"][::-1], resource_for_0_wickets_lost
    )
    
    par_score = runs_scored_by_team_one * (total_resource_available_to_team_two_at_start/total_resource_available_to_team_one_at_start)
    
    return par_score

In [10]:
get_par_score_when_first_innings_is_completed_and_second_innings_is_delayed(
20, 143, 11
)

88.231

### Scenario 4 --> Finding the parscore when First Innings is Cut short and Second innings is completed

In [14]:
def get_par_score_when_first_innings_is_cut_short(
    overs_available_to_team_one_at_start,
    runs_scored_by_team_one,
    wickets_lost_by_team_one,
    overs_used_by_team_one_until_interruption,
    overs_available_to_team_two
):
    resource_for_0_wickets_lost = resource_table_df["0"][::-1]
    
    balls_available_to_team_one_at_start = convert_overs_to_balls(overs_available_to_team_one_at_start)
    balls_used_by_team_one_until_interruption = convert_overs_to_balls(overs_used_by_team_one_until_interruption)
    balls_remaining_to_team_one_during_interruption = balls_available_to_team_one_at_start - balls_used_by_team_one_until_interruption
    balls_available_to_team_two = convert_overs_to_balls(overs_available_to_team_two)
    
    # g 50 score is the estimated score that could had been scored by team one if there were no interruption for TEAM ONE
    # There are many different aspects to be considered to find the g50 score like the team past potential, the kind of pitch they are playing, etc.,
    # For sake of simplicity, we are calculating the g50 score by projecting the team's run rate for the full overs that were alloted to them at start
    g_50_score = (runs_scored_by_team_one/balls_used_by_team_one_until_interruption)*balls_available_to_team_one_at_start
    
    total_resource_available_to_team_one_at_start = np.interp(
        balls_available_to_team_one_at_start, resource_table_df["balls"][::-1], resource_for_0_wickets_lost
    )
    team_one_remaining_resource_during_interruption = np.interp(
        balls_remaining_to_team_one_during_interruption,
        resource_table_df["balls"][::-1],
        resource_table_df[f"{wickets_lost_by_team_one}"][::-1]
    )
    total_resource_used_by_team_one = total_resource_available_to_team_one_at_start - team_one_remaining_resource_during_interruption
    
    total_resource_available_to_team_two = np.interp(
        balls_available_to_team_two, resource_table_df["balls"][::-1], resource_for_0_wickets_lost
    )
    
    par_score = runs_scored_by_team_one + (g_50_score * (total_resource_available_to_team_two - total_resource_used_by_team_one))/100
    
    return par_score

IND vs AUS first t20i 2017

In [12]:
get_par_score_when_first_innings_is_cut_short(
20, 118, 8, 18.4, 6
)

45.85142857142857

AUS vs IND First T20I 2018 Nov

In [15]:
get_par_score_when_first_innings_is_cut_short(
20, 158, 3, 16.1, 16.1
)

173.1158762886598